In [1]:
# MCP Tools in Jupyter Notebooks
# ------------------------------
# IMPORTANT: When using MCP tools in Jupyter notebooks, HTTP-based MCP servers are recommended.
# Stdio-based MCP servers (MCPStdioParams) are now supported with compatibility patches,
# but may have limited functionality. You'll see a warning when using them in Jupyter.

import railtracks as rt
from railtracks.rt_mcp import MCPHttpParams, MCPStdioParams



In [2]:
# Example 1: Using HTTP-based MCP servers (RECOMMENDED for Jupyter)
# -----------------------------------------------------------------
# HTTP-based MCP servers work well in Jupyter notebooks and should be used whenever possible
fetch_server = rt.connect_mcp(MCPHttpParams(url="https://remote.mcpservers.org/fetch/mcp"))
fetch_tools = fetch_server.tools

In [3]:
# Example 2: Using stdio-based MCP servers (NOW SUPPORTED with compatibility patches)
# ---------------------------------------------------------------------------------
# With the new Jupyter compatibility patches, stdio-based MCP servers now work in Jupyter notebooks
# You'll see a warning about potential limitations, but the code should run successfully

# NOTE: Make sure you have `mcp-server-time` installed in your environment: https://pypi.org/project/mcp-server-time/ 
# Define the command and arguments for the stdio-based MCP server
# Note: You'll need to adjust these paths for your environment
MCP_COMMAND = "python"  # Path to your executable
MCP_ARGS = ["-m", "mcp_server_time"]                      # Arguments for the executable

# Create the stdio-based MCP server
# This will show a warning but should work with the compatibility patches
time_stdio_server = rt.connect_mcp(MCPStdioParams(command=MCP_COMMAND, args=MCP_ARGS))
time_stdio_tools = time_stdio_server.tools


In [4]:
# Create a parent tool that can use all the tools from both HTTP and stdio MCP servers
parent_tool = rt.agent_node(
    "Parent Tool",
    tool_nodes={*fetch_tools, *time_stdio_tools},
    system_message=rt.llm.SystemMessage("Provide a response using the tool when asked."),
    llm_model=rt.llm.OpenAILLM("gpt-4o"),
    max_tool_calls=3,
)


In [5]:
with rt.Session(logging_setting="QUIET", timeout=1000):
    response = await rt.call(
        parent_tool,
        user_input="Tell me about conductr.ai. Then, tell me what time it is.",
    )

    print("Response:", response.text)

Using existing session
Response: **About Conductr.ai:**

Conductr is described as the first end-to-end Developer Productivity Engineering (DPE) platform. It is designed to empower developers and teams by delivering pertinent insights to the right person at the right time and place. The platform aims to streamline software development processes through automation, smart insights, and streamlined CI/CD.

Some highlights of Conductr include:
- Integration of quality, security, and performance early in the development cycle.
- Enhanced continuous delivery through efficient and automated workflows.
- Real-time insights and issue detection directly in the developer's IDE.
- Features like Root Cause Agent for identifying errors and providing intelligent fix suggestions.
- Development tools for daily stand-ups, deployment insights, and a velocity dashboard for tracking progress and optimizing workflows.

**Current Time:**

The current time in the America/Los_Angeles timezone is 14:07 (2:07 PM)